In [ ]:
import mysql.connector
from getpass import getpass

def conectar_banco():
    try:
        conn = mysql.connector.connect(
            host='108.167.151.34',
            database='evolud85_idpb',
            user='evolud85_chris',
            password='vGT{R_A^-E+4'
        )
        return conn
    except mysql.connector.Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

def obter_funcoes(email):
    conn = conectar_banco()
    if conn is not None:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT f.Nome_Funcao, COALESCE(c.Numero_Celula, s.Numero_Supervisao, co.Numero_Coordenacao) AS Numero
            FROM Usuarios_X u
            LEFT JOIN Usuario_Funcoes_X uf ON u.ID_Usuario = uf.ID_Usuario
            LEFT JOIN Funcoes_X f ON uf.ID_Funcao = f.ID_Funcao
            LEFT JOIN Celulas_X c ON u.Numero_Celula = c.Numero_Celula AND f.Nome_Funcao = 'Líder'
            LEFT JOIN Supervisao_X s ON u.Numero_Supervisao = s.Numero_Supervisao AND f.Nome_Funcao = 'Supervisor'
            LEFT JOIN Coordenacao_X co ON u.Numero_Coordenacao = co.Numero_Coordenacao AND f.Nome_Funcao = 'Coordenador'
            WHERE u.Email = %s
        """, (email,))
        funcoes = cursor.fetchall()
        cursor.close()
        conn.close()
        return [(funcao[0], funcao[1]) for funcao in funcoes] if funcoes else []
    else:
        return []

def fazer_login(email, senha):
    conn = conectar_banco()
    if conn is not None:
        cursor = conn.cursor()
        cursor.execute("SELECT Senha FROM Usuarios_X WHERE Email = %s", (email,))
        senha_hash = cursor.fetchone()
        cursor.close()
        conn.close()
        
        if senha_hash and senha == senha_hash[0]:  # Simples comparação de senhas; em produção, usar hash
            funcoes = obter_funcoes(email)
            return True, funcoes
        else:
            return False, []
    else:
        return False, []

def main():
    email = input("Digite seu email: ")
    senha = getpass("Digite sua senha: ")  # Usar getpass para esconder a senha ao digitar

    sucesso, funcoes = fazer_login(email, senha)
    if sucesso:
        print("Login bem-sucedido!")
        if funcoes:
            for funcao, numero in funcoes:
                print(f"Você tem permissão de {funcao} da {numero}.")
        else:
            print("Você não tem permissões registradas.")
    else:
        print("Email ou senha incorretos.")

if __name__ == "__main__":
    main()
